In [1]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [2]:
product_data = pd.read_csv('../data/market_integrates_2.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         2.839948         5.853401  4.563120   
1             1         2         2.770059         5.766308  4.808942   
2             1         3         2.343256         5.063622  5.717304   
3             1         4         2.465764         5.219318  5.689583   
4             1         5         2.447657         5.823841  4.780318   
..          ...       ...              ...              ...       ...   
995         100         6         2.876320         5.018660  5.117055   
996         100         7         2.971446         5.675144  4.180734   
997         100         8         2.136148         5.015674  5.799872   
998         100         9         2.775236         5.205261  5.208689   
999         100        10         2.976228         5.566211  4.526664   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0         3.260590  0.034609  0.045079  1.399477

In [3]:
# # Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
# product_data_complete = product_data.rename(columns={'labor':'supply_instrument0',
#                                                     #   'capital':'supply_instruments1',
#                                                         # 'marginal_cost': 'supply_instrument2'
#                                                         })



In [4]:
# the endog regressor
X1_formulation = pyblp.Formulation('1  + prices+ characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

# supply side charact 
# X3_formulation = pyblp.Formulation('1 + log(labor) + log(capital)')

# formulation for the cost thing (assume cost fomrulation being labor and capital)
product_formulations = (X1_formulation,
                         X2_formulation
                           #  X3_formulation
                           )
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [5]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [6]:
mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration, add_exogenous=True)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


In [7]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [8]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           699         2194         0     +1.337254E-20                 +1.185642E-09  +1.000000E+00

Optimization completed after 00:00:01.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:02.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------